## Passing the password

There are couple of ways to pass the password:
- Interactively - When using the front applications interactively
- Environment variable - When writing a quick script
- .pgpass File - In production (because environment variables can be accessed with ps)
- Environment + .pgpass - In our course

In [ ]:
psql --help

In [ ]:
export PGPASSWORD=cool-postgres-course
psql postgresql://postgres@db:5432 -c "SELECT 1 AS EXAMPLE"
# Remember to restart kernel or the PGPASSWORD will be

In [ ]:
# Since the docker file already got a .pgpass (how convinient) we would want to demonstrate a new environment
rm ~/.pgpass # Remove existing .pgpass file
touch ~/.pgpass
echo db:5432:*:postgres:cool-postgres-course > ~/.pgpass
chmod 600 ~/.pgpass
chown jovyan ~/.pgpass
psql -h db -U postgres -c "SELECT 1 AS EXAMPLE"

In [ ]:
export PGHOST=db
export PGUSER=postgres
export PGDATABASE=postgres
psql -c "SELECT 1 AS EXAMPLE" # No need for any params!

## PSQL

### Scripting style

#### Using multiple commands in a long string

In [ ]:
psql << EOM
    SELECT 1 AS first;
    SELECT 1 AS some, 2 AS more, 3 AS example;
EOM

#### Using multiple commands in different psql instances

In [ ]:
psql << EOM
    SELECT 1 AS first;
EOM

psql << EOM
    SELECT 1 AS some, 2 AS more, 3 AS example;
EOM

#### Let's create some data

In [ ]:
psql << EOM
    DROP DATABASE IF EXISTS newdb;
    CREATE DATABASE newdb;
EOM
export PGDATABASE=newdb

In [ ]:
psql << EOM
    DROP TABLE IF EXISTS PRODUCT;
    CREATE TABLE IF NOT EXISTS PRODUCT (
      id   SERIAL,
      name VARCHAR(100) NOT NULL,
      sku  CHAR(8)
    );
EOM

psql << EOM
    INSERT INTO product VALUES
    (DEFAULT, 'Banana', '1'),
    (DEFAULT, 'Orange', '2'),
    (DEFAULT, 'Milki', '3')
    ;
EOM

#### Using meta commands
We already saw the `\c` command, let's some other interesting ones

In [ ]:
psql -c "\x off"

In [ ]:
# List databases
psql -c "\l+"

In [ ]:
# List relations
psql -c "\d+"

In [ ]:
# Specific relation data
psql -c "\d+ product"

In [ ]:
# Roles
psql -c "\du"

In [ ]:
# Time queries
psql << EOM
    \timing
    SELECT COUNT(*) product;
    SELECT COUNT('id') product;
    \timing
EOM

In [ ]:
# Access methods
psql -c "\dA+"

In [ ]:
# Table spaces
psql -c "\db+"

In [ ]:
# Variables
psql << EOM
    \set we_learn postgres
    \echo What do we learn? :we_learn!
EOM

In [ ]:
# Variables with SQL queries
psql << EOM
    \set my_table product
    SELECT * FROM :my_table
EOM

#### Cool and relatively easy looping with \gexec

In [ ]:
psql << EOM    
    SELECT FORMAT('SELECT %s AS example', id) FROM generate_series(1,4) AS id
    \gexec
EOM

So how does it work?
1. Query a dynamic SQL query string

In [ ]:
psql -c "SELECT 'SELECT 1 AS example' unimportant_column_name;"

2. Create multiple strings

In [ ]:
psql -c "SELECT FORMAT('SELECT %s AS example', id) FROM generate_series(1,4) AS id;"

3. Run a dynamic query with `\gexec` which is actually executing the `query buffer`

In [ ]:
psql << EOM    
    SELECT 'SELECT 1 AS example' unimportant_column_name
    \gexec
EOM

4. Run the multiple dynamic strings

In [ ]:
psql << EOM    
    SELECT FORMAT('SELECT %s AS example', id) FROM generate_series(1,4) AS id
    \gexec
EOM

Pretty cool! \
Can be used in a wide variety of use cases, just so you get a taste of it:

In [ ]:
psql << EOM    
    SELECT format('CREATE INDEX ON product(%I)', attname)
    FROM pg_attribute
    WHERE attrelid = 'product'::regclass AND attnum > 0
    ORDER BY attnum
    \gexec
EOM

In [ ]:
psql -c "\d product"

## More useful tools

**clusterdb** — cluster a PostgreSQL database \
**createdb** — create a new PostgreSQL database \
**createuser** — define a new PostgreSQL user account \
**dropdb** — remove a PostgreSQL database \
**dropuser** — remove a PostgreSQL user account \
**pg_amcheck** — checks for corruption in one or more PostgreSQL databases \
**pg_basebackup** — take a base backup of a PostgreSQL cluster \
**pgbench** — run a benchmark test on PostgreSQL \
**pg_config** — retrieve information about the installed version of PostgreSQL \
**pg_dump** — extract a PostgreSQL database into a script file or other archive file \
**pg_dumpall** — extract a PostgreSQL database cluster into a script file \
**pg_isready** — check the connection status of a PostgreSQL server \
**pg_receivewal** — stream write-ahead logs from a PostgreSQL server \
**pg_restore** — restore a PostgreSQL database from an archive file created by pg_dump \
**pg_verifybackup** — verify the integrity of a base backup of a PostgreSQL cluster \
**reindexdb** — reindex a PostgreSQL database \
**vacuumdb** — garbage-collect and analyze a PostgreSQL database